# Data Preprocessing

In this notebook we will preprocess our reviews and remove unnecessary strings.


In [1]:
import re
import os
import wandb
import pandas as pd
from typing import Optional
from dotenv import load_dotenv
from arabert.preprocess import ArabertPreprocessor
from sklearn.model_selection import train_test_split

from huggingface_hub import login
from datasets import load_dataset

from rich import print
from pathlib import Path
from tqdm.auto import tqdm
from collections import defaultdict

In [2]:
tqdm.pandas()
load_dotenv()
wandb.login()
login(token=os.getenv('HF_TOKEN'),
      add_to_git_credential=True, write_permission=True)  # HF

wandb: Currently logged in as: e_hossam96. Use `wandb login --relogin` to force relogin


Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /home/hossam/.cache/huggingface/token
Login successful


In [3]:
# wandb configs
PROJECT_NAME = 'ARABIC_DISAMBIGUATION_STUDY'
JOB_TYPE = 'DATA_PREPROCESSING'
TAGS = ['ARABERT_PREPROCESSOR', 'FARASAPY', 'REGEX', 'TOPIC']
NOTES = 'Preprocessing data for topic classification with disambiguation'
RUN_NAME = 'Data_Preprocessing_SANAD'
config = defaultdict(dict)

In [32]:
seed = 42
arabert_model = 'aubmindlab/bert-base-arabertv2'
# data_path = Path('../data/ArSen-20_publish.csv')
data_ckpt = 'arbml/SANAD'
processed_data_path = Path('../data/processed_data/sanad/')

In [5]:
# if data_path.suffix == '.csv':
#     data = pd.read_csv(data_path)

In [6]:
data = load_dataset(data_ckpt, trust_remote_code=True)
data

Repo card metadata block was not found. Setting CardData to empty.


DatasetDict({
    train: Dataset({
        features: ['Article', 'label'],
        num_rows: 141807
    })
})

In [7]:
print(data['train'][0])

{
    'Article': 'يحيي كل من العروسي وعواطف وعمار والعنبري أمجاد الأغنية المغربية العصرية والكلاسيكية، في سهرة تنظم 
يوم 26 ماي المقبل، بمركب مولاي عبد الله. من جهته، يتقاسم الفنان الشاب حاتم عمور منصة النهضة المخصصة للأغنية 
الشرقية، مع ألمع نجوم الطرب العربي، ليروي عطش عشاق الأغنية المغربية الأصيلة، في سهرة تقام في 31 ماي. وتتوقع "مغرب 
الثقافات" إقبالا جماهيريا كبيرا على السهرتين المغربيتين خلال هذه الدورة المنظمة من 24 ماي إلى فاتح يونيو المقبلين، 
بفضل شعبية النجوم الخمسة، سواء المغنية الاستثنائية عواطف، التي تحدّت الإعاقة واستحوذت على حب الجمهور وإعجابه بقدرتها
على الغناء بخمس لغات، وفاطمة الزهراء العروسي، الملقبة بـ"زهرة الطرب"، التي ولجت عالم الفن وتألقت فيه في سن مبكرة، 
بالإضافة إلى محمد العنبري، ابن مدينة فاس، الحاصل على جوائز وطنية ودولية عدة، وعتيقة عمار، كواحدة من أعمدة الأغنية 
المغربية، وأخيرا، المغني الشاب حاتم عمور، الذي اختبر دعم الجمهور وحبه في أكثر من مناسبة، وأهمها مشاركته في برنامج 
المسابقات الغنائية "استوديو دوزيم". وفي إطار الدعم، الذي توليه إدارة مهرجان "موازين.. إيقاعات العالم" للأغنية 
المغربية بمختلف أصنافها، أعلنت "مغرب الثقافات"، في وقت سابق، عن مشاركة كل من مجموعات "السهام"، و"جيل الغيوان"، 
و"غيوان سلوان" بمنصة المركب الرياضي الأمير مولاي عبد الله، كما سيلتقي عشاق "كناوة والبلوز" بالمنصة ذاتها، مع وصلات 
غنائية للفنانين جبارة ومجيد بقاس يوم 25 ماي المقبل.',
    'label': 5
}

In [8]:
label_names = data['train'].features['label'].names
print(label_names)

['Tech', 'Finance', 'Politics', 'Religion', 'Medical', 'Culture', 'Sports']

In [9]:
data = data['train'].to_pandas()
data.head(2)

,Article,label
0,يحيي كل من العروسي وعواطف وعمار والعنبري أمجاد...,5
1,أخبارنا المغربية ـ هدى جميعي\nتحول فنان مغربي ...,5


In [10]:
data['label'] = data.label.progress_apply(lambda x: label_names[x])

  0%|          | 0/141807 [00:00<?, ?it/s]

In [11]:
data.head(2)

,Article,label
0,يحيي كل من العروسي وعواطف وعمار والعنبري أمجاد...,Culture
1,أخبارنا المغربية ـ هدى جميعي\nتحول فنان مغربي ...,Culture


In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141807 entries, 0 to 141806
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   Article  141807 non-null  object
 1   label    141807 non-null  object
dtypes: object(2)
memory usage: 2.2+ MB


In [13]:
print(data.label.value_counts(normalize=True).to_dict())

{
    'Finance': 0.2775321387519657,
    'Sports': 0.2084382294244995,
    'Politics': 0.14441459166331705,
    'Medical': 0.13713709478375538,
    'Tech': 0.1318623199136855,
    'Religion': 0.05304392589928565,
    'Culture': 0.04757169956349122
}

In [14]:
# select subsample of the data
_, data = train_test_split(data, test_size=10_000, random_state=seed, stratify=data.label.to_list())
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 57159 to 120789
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Article  10000 non-null  object
 1   label    10000 non-null  object
dtypes: object(2)
memory usage: 234.4+ KB


In [15]:
data.head(2)

,Article,label
57159,كثيرا ما ربطت دراسات عديدة بين تناول المكسرات ...,Medical
79089,أخبارنا:الرباط\nمن المرتقب أن تعيد الدولة المغ...,Politics


In [16]:
data = data.rename(columns={'Article': 'text'}).reset_index(drop=True)

In [17]:
i = 100  # 14
print(data.loc[i, 'text'])

مبديع : نسبة المشاركة في الإضراب الوطني الأخير  كانت في حدود 22 في المائة

 
قال السيد محمد مبديع الوزير المنتدب لدى رئيس الحكومة المكلف بالوظيفة العمومية وتحديث الإدارة، اليوم الخميس، إن 
الإضراب الوطني الاخير بالإدارات العمومية والجماعات الترابية "مر في ظروف مسؤولة وجد عادية"، مضيفا أن نسبة المشاركة 
كانت في حدود 22 في المائة.وجاء في بلاغ تلاه السيد مصطفى الخلفي وزير الاتصال الناطق الرسمي باسم الحكومة خلال ندوة 
صحافية عقب انعقاد مجلس الحكومة أن السيد مبديع أكد في عرض حول الإضراب الوطني بالإدارات العمومية والجماعات الترابية 
ليوم الثلاثاء الماضي على "أن هذا الإضراب مر في ظروف مسؤولة وجد عادية، دون أن تسجل أي تجاوزات أو اختلالات في السير 
العادي للمرافق العمومية" . وأبرز السيد مبديع في هذا العرض أن الحكومة عملت على عدم الدخول في جدل الأرقام وعدم 
التعليق على الأرقام التي قدمت من طرف الهيئات النقابية التي دعت الى الإضراب.

In [18]:
arabert_prep = ArabertPreprocessor(model_name=arabert_model)

[2024-09-14 02:27:20,822 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


In [19]:
print(arabert_prep.preprocess(data.loc[i, 'text']))

مبديع : نسب +ة ال+ مشارك +ة في ال+ إضراب ال+ وطني ال+ أخير كان +ت في حدود 22 في ال+ مائ +ة قال ال+ سيد محمد مبديع 
ال+ وزير ال+ منتدب لدى رئيس ال+ حكوم +ة ال+ مكلف ب+ ال+ وظيف +ة ال+ عمومي +ة و+ تحديث ال+ إدار +ة ، ال+ يوم ال+ 
خميس ، إن ال+ إضراب ال+ وطني ال+ أخير ب+ ال+ إدار +ات ال+ عمومي +ة و+ ال+ جماع +ات ال+ ترابي +ة " مر في ظروف مسؤول 
+ة وجد عادي +ة " ، مضيف +ا أن نسب +ة ال+ مشارك +ة كان +ت في حدود 22 في ال+ مائ +ة . و+ جاء في بلاغ تلا +ه ال+ سيد 
مصطفى ال+ خلفي وزير ال+ اتصال ال+ ناطق ال+ رسمي باسم ال+ حكوم +ة خلال ندو +ة صحافي +ة عقب انعقاد مجلس ال+ حكوم +ة 
أن ال+ سيد مبديع أكد في عرض حول ال+ إضراب ال+ وطني ب+ ال+ إدار +ات ال+ عمومي +ة و+ ال+ جماع +ات ال+ ترابي +ة ل+ يوم
ال+ ثلاثاء ال+ ماضي على " أن هذا ال+ إضراب مر في ظروف مسؤول +ة وجد عادي +ة ، دون أن تسجل أي تجاوز +ات أو اختلال +ات
في ال+ سير ال+ عادي ل+ ال+ مرافق ال+ عمومي +ة " . و+ أبرز ال+ سيد مبديع في هذا ال+ عرض أن ال+ حكوم +ة عمل +ت على 
عدم ال+ دخول في جدل ال+ أرقام و+ عدم ال+ تعليق على ال+ أرقام التي قدم +ت من طرف ال+ هيئ +ات ال+ نقابي +ة التي دع +ت
الى ال+ إضراب .

In [20]:
def clean_text(tweet: str, is_arabert: Optional[bool] = False) -> str:
    '''Preprocess and clean text using Farasapy and regex rules.'''
    text = arabert_prep.preprocess(tweet)
    if not is_arabert:
        text = arabert_prep.unpreprocess(text)
    # regex patterns
    link = r'\[رابط\]'
    user = r'\[مستخدم\]'
    mail = r'\[بريد\]'
    underscore = r'_'
    hashtag = r'#'
    space = r'\s+'
    patterns = [link, user, mail, underscore, hashtag, space]
    for p in patterns:
        text = re.sub(p, ' ', text)
    return text

In [21]:
print(clean_text(data.loc[i, 'text']))

مبديع: نسبة المشاركة في الإضراب الوطني الأخير كانت في حدود 22 في المائة قال السيد محمد مبديع الوزير المنتدب لدى 
رئيس الحكومة المكلف بالوظيفة العمومية وتحديث الإدارة، اليوم الخميس، إن الإضراب الوطني الأخير بالإدارات العمومية 
والجماعات الترابية "مر في ظروف مسؤولة وجد عادية"، مضيفا أن نسبة المشاركة كانت في حدود 22 في المائة. وجاء في بلاغ 
تلاه السيد مصطفى الخلفي وزير الاتصال الناطق الرسمي باسم الحكومة خلال ندوة صحافية عقب انعقاد مجلس الحكومة أن السيد 
مبديع أكد في عرض حول الإضراب الوطني بالإدارات العمومية والجماعات الترابية ليوم الثلاثاء الماضي على "أن هذا الإضراب 
مر في ظروف مسؤولة وجد عادية، دون أن تسجل أي تجاوزات أو اختلالات في السير العادي للمرافق العمومية". وأبرز السيد 
مبديع في هذا العرض أن الحكومة عملت على عدم الدخول في جدل الأرقام وعدم التعليق على الأرقام التي قدمت من طرف الهيئات 
النقابية التي دعت الى الإضراب.

In [23]:
data['processed_text'] = data.text.progress_apply(clean_text, is_arabert=False)

  0%|          | 0/10000 [00:00<?, ?it/s]

In [24]:
data['processed_text_arabert'] = data.text.progress_apply(clean_text, is_arabert=True)

  0%|          | 0/10000 [00:00<?, ?it/s]

In [25]:
for k, v in data.loc[i].to_dict().items():
    print(f'{k}:\n{v}')

text:
مبديع : نسبة المشاركة في الإضراب الوطني الأخير  كانت في حدود 22 في المائة

 
قال السيد محمد مبديع الوزير المنتدب لدى رئيس الحكومة المكلف بالوظيفة العمومية وتحديث الإدارة، اليوم الخميس، إن 
الإضراب الوطني الاخير بالإدارات العمومية والجماعات الترابية "مر في ظروف مسؤولة وجد عادية"، مضيفا أن نسبة المشاركة 
كانت في حدود 22 في المائة.وجاء في بلاغ تلاه السيد مصطفى الخلفي وزير الاتصال الناطق الرسمي باسم الحكومة خلال ندوة 
صحافية عقب انعقاد مجلس الحكومة أن السيد مبديع أكد في عرض حول الإضراب الوطني بالإدارات العمومية والجماعات الترابية 
ليوم الثلاثاء الماضي على "أن هذا الإضراب مر في ظروف مسؤولة وجد عادية، دون أن تسجل أي تجاوزات أو اختلالات في السير 
العادي للمرافق العمومية" . وأبرز السيد مبديع في هذا العرض أن الحكومة عملت على عدم الدخول في جدل الأرقام وعدم 
التعليق على الأرقام التي قدمت من طرف الهيئات النقابية التي دعت الى الإضراب.

label:
Finance

processed_text:
مبديع: نسبة المشاركة في الإضراب الوطني الأخير كانت في حدود 22 في المائة قال السيد محمد مبديع الوزير المنتدب لدى 
رئيس الحكومة المكلف بالوظيفة العمومية وتحديث الإدارة، اليوم الخميس، إن الإضراب الوطني الأخير بالإدارات العمومية 
والجماعات الترابية "مر في ظروف مسؤولة وجد عادية"، مضيفا أن نسبة المشاركة كانت في حدود 22 في المائة. وجاء في بلاغ 
تلاه السيد مصطفى الخلفي وزير الاتصال الناطق الرسمي باسم الحكومة خلال ندوة صحافية عقب انعقاد مجلس الحكومة أن السيد 
مبديع أكد في عرض حول الإضراب الوطني بالإدارات العمومية والجماعات الترابية ليوم الثلاثاء الماضي على "أن هذا الإضراب 
مر في ظروف مسؤولة وجد عادية، دون أن تسجل أي تجاوزات أو اختلالات في السير العادي للمرافق العمومية". وأبرز السيد 
مبديع في هذا العرض أن الحكومة عملت على عدم الدخول في جدل الأرقام وعدم التعليق على الأرقام التي قدمت من طرف الهيئات 
النقابية التي دعت الى الإضراب.

processed_text_arabert:
مبديع : نسب +ة ال+ مشارك +ة في ال+ إضراب ال+ وطني ال+ أخير كان +ت في حدود 22 في ال+ مائ +ة قال ال+ سيد محمد مبديع 
ال+ وزير ال+ منتدب لدى رئيس ال+ حكوم +ة ال+ مكلف ب+ ال+ وظيف +ة ال+ عمومي +ة و+ تحديث ال+ إدار +ة ، ال+ يوم ال+ 
خميس ، إن ال+ إضراب ال+ وطني ال+ أخير ب+ ال+ إدار +ات ال+ عمومي +ة و+ ال+ جماع +ات ال+ ترابي +ة " مر في ظروف مسؤول 
+ة وجد عادي +ة " ، مضيف +ا أن نسب +ة ال+ مشارك +ة كان +ت في حدود 22 في ال+ مائ +ة . و+ جاء في بلاغ تلا +ه ال+ سيد 
مصطفى ال+ خلفي وزير ال+ اتصال ال+ ناطق ال+ رسمي باسم ال+ حكوم +ة خلال ندو +ة صحافي +ة عقب انعقاد مجلس ال+ حكوم +ة 
أن ال+ سيد مبديع أكد في عرض حول ال+ إضراب ال+ وطني ب+ ال+ إدار +ات ال+ عمومي +ة و+ ال+ جماع +ات ال+ ترابي +ة ل+ يوم
ال+ ثلاثاء ال+ ماضي على " أن هذا ال+ إضراب مر في ظروف مسؤول +ة وجد عادي +ة ، دون أن تسجل أي تجاوز +ات أو اختلال +ات
في ال+ سير ال+ عادي ل+ ال+ مرافق ال+ عمومي +ة " . و+ أبرز ال+ سيد مبديع في هذا ال+ عرض أن ال+ حكوم +ة عمل +ت على 
عدم ال+ دخول في جدل ال+ أرقام و+ عدم ال+ تعليق على ال+ أرقام التي قدم +ت من طرف ال+ هيئ +ات ال+ نقابي +ة التي دع +ت
الى ال+ إضراب .

In [26]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   text                    10000 non-null  object
 1   label                   10000 non-null  object
 2   processed_text          10000 non-null  object
 3   processed_text_arabert  10000 non-null  object
dtypes: object(4)
memory usage: 312.6+ KB


In [27]:
train, valid = train_test_split(
    data, test_size=0.2, stratify=data.label.to_list(), random_state=seed)
valid, test = train_test_split(
    valid, test_size=0.5, stratify=valid.label.to_list(), random_state=seed)

In [28]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8000 entries, 3745 to 4805
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   text                    8000 non-null   object
 1   label                   8000 non-null   object
 2   processed_text          8000 non-null   object
 3   processed_text_arabert  8000 non-null   object
dtypes: object(4)
memory usage: 312.5+ KB


In [29]:
valid.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 372 to 4472
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   text                    1000 non-null   object
 1   label                   1000 non-null   object
 2   processed_text          1000 non-null   object
 3   processed_text_arabert  1000 non-null   object
dtypes: object(4)
memory usage: 39.1+ KB


In [30]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 3050 to 8458
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   text                    1000 non-null   object
 1   label                   1000 non-null   object
 2   processed_text          1000 non-null   object
 3   processed_text_arabert  1000 non-null   object
dtypes: object(4)
memory usage: 39.1+ KB


In [31]:
print(train.label.value_counts(normalize=True).to_dict())

{
    'Finance': 0.2775,
    'Sports': 0.208375,
    'Politics': 0.144375,
    'Medical': 0.137125,
    'Tech': 0.131875,
    'Religion': 0.053125,
    'Culture': 0.047625
}

In [33]:
data_artifact = wandb.Artifact(
    name='Topic-Classification-Dataset', type='dataset')

data_dict = {'train': train, 'valid': valid, 'test': test}

for n, d in data_dict.items():
    processed_data_path.mkdir(exist_ok=True)
    split_path = processed_data_path.joinpath(f'{n}.csv')
    config['data'][n] = {
        'size': len(d),
        'local_path': split_path,
        'label_distribution': d.label.value_counts(normalize=True).to_dict(),
    }
    d.to_csv(split_path, index=False)
    data_artifact.add_file(local_path=split_path, name=n)

In [34]:
print(config)

defaultdict(<class 'dict'>, {
    'data': {
        'train': {
            'size': 8000,
            'local_path': PosixPath('../data/processed_data/sanad/train.csv'),
            'label_distribution': {
                'Finance': 0.2775,
                'Sports': 0.208375,
                'Politics': 0.144375,
                'Medical': 0.137125,
                'Tech': 0.131875,
                'Religion': 0.053125,
                'Culture': 0.047625
            }
        },
        'valid': {
            'size': 1000,
            'local_path': PosixPath('../data/processed_data/sanad/valid.csv'),
            'label_distribution': {
                'Finance': 0.278,
                'Sports': 0.209,
                'Politics': 0.144,
                'Medical': 0.137,
                'Tech': 0.132,
                'Religion': 0.053,
                'Culture': 0.047
            }
        },
        'test': {
            'size': 1000,
            'local_path': PosixPath('../data/processed_data/sanad/test.csv'),
            'label_distribution': {
                'Finance': 0.277,
                'Sports': 0.208,
                'Politics': 0.145,
                'Medical': 0.137,
                'Tech': 0.132,
                'Religion': 0.053,
                'Culture': 0.048
            }
        }
    }
})

In [35]:
run = wandb.init(project=PROJECT_NAME, job_type=JOB_TYPE,
                 name=RUN_NAME, notes=NOTES, tags=TAGS, config=config)

In [36]:
_ = run.log_artifact(data_artifact)

In [37]:
run.finish()